In [ ]:
#!pip install nltk
#!pip install evaluate

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import movie_reviews

In [3]:
import nltk
from nltk.corpus import movie_reviews

# 데이터 다운로드
nltk.download("movie_reviews")

# 영화 리뷰 데이터를 준비
reviews = [movie_reviews.raw(fid) for fid in movie_reviews.fileids()]

# 첫 번째 리뷰와 라벨 출력
print("첫 번째 리뷰 내용:")
print(reviews[0])
print("첫 번째 리뷰 라벨:", movie_reviews.categories(movie_reviews.fileids()[0]))


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\human\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


첫 번째 리뷰 내용:
plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member 

In [4]:
print(movie_reviews.fileids())

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt', 'neg/cv010_29063.txt', 'neg/cv011_13044.txt', 'neg/cv012_29411.txt', 'neg/cv013_10494.txt', 'neg/cv014_15600.txt', 'neg/cv015_29356.txt', 'neg/cv016_4348.txt', 'neg/cv017_23487.txt', 'neg/cv018_21672.txt', 'neg/cv019_16117.txt', 'neg/cv020_9234.txt', 'neg/cv021_17313.txt', 'neg/cv022_14227.txt', 'neg/cv023_13847.txt', 'neg/cv024_7033.txt', 'neg/cv025_29825.txt', 'neg/cv026_29229.txt', 'neg/cv027_26270.txt', 'neg/cv028_26964.txt', 'neg/cv029_19943.txt', 'neg/cv030_22893.txt', 'neg/cv031_19540.txt', 'neg/cv032_23718.txt', 'neg/cv033_25680.txt', 'neg/cv034_29446.txt', 'neg/cv035_3343.txt', 'neg/cv036_18385.txt', 'neg/cv037_19798.txt', 'neg/cv038_9781.txt', 'neg/cv039_5963.txt', 'neg/cv040_8829.txt', 'neg/cv041_22364.txt', 'neg/cv042_11927.txt', 'neg/cv043_16808.t

In [21]:
type(reviews)

list

In [11]:
movie_reviews.categories()

['neg', 'pos']

In [95]:
labels = [movie_reviews.categories(fid) for fid in movie_reviews.fileids()]

In [97]:
#labels #리스트의 리스트 형식

In [23]:
review_array = np.array(reviews)
labels_array = np.array(labels)

In [38]:
np.sum(labels_array == "pos"), np.sum(labels_array == "neg")

(1000, 1000)

In [73]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(reviews, labels, random_state=42)

In [74]:
type(y_train)

list

In [75]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_inputs = tokenizer(x_train, truncation=True, padding=True, return_tensors="pt")
test_inputs = tokenizer(x_test, truncation=True, padding=True, return_tensors="pt")

In [76]:
train_inputs

{'input_ids': tensor([[ 101, 2182, 1005,  ..., 4110, 2011,  102],
        [ 101, 1996, 2236,  ..., 1012, 2002,  102],
        [ 101, 4626, 2520,  ..., 1996, 4637,  102],
        ...,
        [ 101, 1045, 3984,  ..., 2009, 2453,  102],
        [ 101, 1996, 3614,  ...,    0,    0,    0],
        [ 101, 1045, 2031,  ..., 8197, 4892,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [77]:
type(y_train)

list

In [83]:
import torch

class RevuewDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.label_mapping = {'neg': 0, 'pos': 1}
        self.labels = [self.label_mapping[label[0]] for label in labels]

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        input = {key: torch.tensor(value) for key, value in self.inputs.items()}
        input["label"] = torch.tensor(self.labels[idx])
        return input

In [84]:
train_dataset = RevuewDataset(train_inputs, y_train)
test_dataset = RevuewDataset(test_inputs, y_test)

In [107]:
for input in train_dataset:
    print(input.keys())
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16696\3797335270.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = {key: torch.tensor(value) for key, value in self.inputs.items()}


In [56]:
import torch

device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
print(device)


cpu


In [63]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

from evaluate import load

import numpy as np


In [64]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [109]:
metric = load("accuracy")

def compute_metrics(evaluation_dataset):
    logits, labels = evaluation_dataset
    predictions = np.argmax(logits, axis=-1)
    return { "accuracy" : (predictions == labels).sum()}

In [112]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments


train_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps = 500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model = model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    args=train_args
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`